# Load And Analysis Data

In [3]:
import pickle


data = pickle.load(open("data/train_data.pkl", 'rb'))

In [4]:
data[1]

("Harini Komaravelli Test Analyst at Oracle, Hyderabad  Hyderabad, Telangana - Email me on Indeed: indeed.com/r/Harini- Komaravelli/2659eee82e435d1b  ➢ 6 Yrs. of IT Experience in Manual and Automation testing.  WORK EXPERIENCE  QA Analyst  Oracle  Test Analyst at Oracle, Hyderabad  Infosys Ltd -  Hyderabad, Telangana -  November 2011 to February 2016  Hyderabad from Nov 2011 to Feb17 2016 ➢ Worked in Tata Consultancy Services, Hyderabad from Feb 24 to Apr 11 2017 ➢ Currently working as a Test Analyst at Oracle, Hyderabad  QA Analyst with 6 years of IT experience  Oracle  EDUCATION  MCA  Osmania University  B.Sc. in Computer Science  Osmania University  SKILLS  Functional Testing, Blue Prism, Qtp  ADDITIONAL INFORMATION  Area of Expertise:  ➢ Familiar with Agile Methodologies. ➢ Having knowledge in Energy (Petroleum) & Health Care domains. ➢ Involved in preparation of Test Scenarios. ➢ Preparing Test Data for the test cases.  https://www.indeed.com/r/Harini-Komaravelli/2659eee82e435d1b?

# Create and train nlp model

In [33]:
import spacy
import random
import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding


nlp = spacy.blank("en")


def train_model(train_data):
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
        
        
        for _, anatotation in train_data:
            for ent in anatotation['entities']:
                ner.add_label(ent[2])
        
        
    pipe_exceptions = ["ner"]
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
    with nlp.disable_pipes(*other_pipes):  # only train NER
        # reset and initialize the weights randomly – but only if we're
        # training a new model
        opt = nlp.begin_training()
        for itn in range(10):
            print("Strt iter: "+str(itn))
            random.shuffle(train_data)
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(train_data, size=compounding(4.0, 32.0, 1.001))
            for texts, annotations  in train_data:
                try:
                    nlp.update(
                    [texts],  # batch of texts
                    [annotations],  # batch of annotations
                    drop=0.3,  # dropout - make it harder to memorise data
                    losses=losses,
                    sgd=opt)
                except Exception as e:
                    #print(e)
                    pass
                
            print("Losses", losses)        

In [34]:
train_model(train_data=data)

Strt iter: 0
Losses {'ner': 12910.276924661317}
Strt iter: 1
Losses {'ner': 9490.533529570856}
Strt iter: 2
Losses {'ner': 8509.357369359304}
Strt iter: 3
Losses {'ner': 8100.6438359973445}
Strt iter: 4
Losses {'ner': 7122.133079482255}
Strt iter: 5
Losses {'ner': 7863.632327497876}
Strt iter: 6
Losses {'ner': 6272.576605055368}
Strt iter: 7
Losses {'ner': 5664.183539927051}
Strt iter: 8
Losses {'ner': 5659.924423979124}
Strt iter: 9
Losses {'ner': 4956.2267034108345}


# Save NLP Model and Reload

In [37]:
!mkdir model

In [38]:
nlp.to_disk("model/ner_model")

In [41]:
!ls model/ner_model

meta.json  ner	tokenizer  vocab


In [42]:
reloaded = spacy.load('model/ner_model')

# Test Model

In [49]:
cv = "Ramesh HP CES ASSOCIATE CONSULTANT  Bangalore, Karnataka - Email me on Indeed: indeed.com/r/Ramesh-HP/95fc615713630c4e  • 4 years of experience in engineering Technology software sales and strategic sourcing in B2B platform. • Proven track record of generating increased revenue by involving in professional sales strategies. • Responsible for software installation, network configuration, application integration with existing system and Technical support. • Effectively work with cross functional teams to deliver right solutions to client's requirements. • Complete involvement in client meetings with respect to requirement collection, suggesting solutions and financial negotiations. • Good experience in account management, having a track record of generating repeated business. • Responsible for report generation with respect qualified leads and expected commitments in closing deals. • Worked on a multiple market sector, responsibility to manage sector wise market analysis and drive business parallely.  Willing to relocate: Anywhere  WORK EXPERIENCE  CES ASSOCIATE CONSULTANT  SAP ARIBA -  November 2016 to October 2017  • Responsible for supplier management via Ariba Discovery Which is B2B consulting platform. • Strategic sourcing of supplier corresponding to the buyer's products & service based commodities globally. • Effectively analyse and conduct commodity research on project description from the buyer's postings. • Effective handling of multiple projects and converting potential leads into revenue for Ariba. • Having track record of maintaining 100% revenue target by monthly & quarterly.  SALES ENGINEER - CONCEPT TECHNOLOGY SOLUTION  -  January 2014 to November 2016  • Effective selling of CAD, CAM & Analysis Software's. Which has got multiple market sector. • Generating qualified leads by sourcing market, sector wise & implementing sales action plans. • Giving presentation about the company, products & service offers. • Identifying the client requirements, Plan for proposing a solution by integrating with internal technical team. • Provide value addition to the prospect by involving my superiors with client management team.  https://www.indeed.com/r/Ramesh-HP/95fc615713630c4e?isid=rex-download&ikw=download-top&co=IN   • Responsible to make negotiations and up closing the deals. • Provide technical support after sales and maintain healthy relationship with the prospect. • Responsible to achieve targets monthly, quarterly & annually.  EDUCATION  MCA in COMPUTER APPLICATION  Dayananda Sagar College of Engineering -  Bengaluru, Karnataka  BACHELOR OF SCIENCE in Electronics  Govt Science College -  Hassan, Karnataka  SKILLS  Lead genearation, Customer Handling, cold calling, Negotiation, upselling, IT sales, outbound calling, Technical Support, sales forcasting, Software sale, product demonstration, cross selling, Inside Sales, Technical sales, MS office, software integration, Network Management  ADDITIONAL INFORMATION  • Excellent Communication both verbal & written MS-Office • Sales Forecasting SAP Business Objective Tool • Strategic Prospecting Ariba Network Admin Tool • Product Knowledge Ariba B2B Cloud Platform • Social Networking • Negotiation • Customer Relationship Management • Technical Support"

In [55]:
doc = nlp(cv)


# [(ent.label_, ent.text) for ent in doc.ents]

for ent in doc.ents:
    print(f'{ent.label_ :{30}}- {ent.text}')


Name                          - Ramesh HP
Designation                   - CES ASSOCIATE CONSULTANT
Location                      - Bangalore
Email Address                 - indeed.com/r/Ramesh-HP/95fc615713630c4e
Designation                   - CES ASSOCIATE CONSULTANT
Companies worked at           - SAP ARIBA
Degree                        - MCA in COMPUTER APPLICATION
College Name                  - Dayananda Sagar College of Engineering
Degree                        - BACHELOR OF SCIENCE in Electronics
College Name                  - Govt Science College
Skills                        - Lead genearation, Customer Handling, cold calling, Negotiation, upselling, IT sales, outbound calling, Technical Support, sales forcasting, Software sale, product demonstration, cross selling, Inside Sales, Technical sales, MS office, software integration, Network Management
